In [1]:
import pickle
import pandas as pd
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.tokenize import RegexpTokenizer
import re
import nltk
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import itertools
import string
from tqdm import tqdm
from sklearn import datasets
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/ian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
with open("alaska_hotels_redux.pickle", "rb") as to_read:
    updated_early_df = pickle.load(to_read)

In [6]:
with open("alaska_hotels_first_attempt.pickle", "rb") as to_read:
    hotels_df = pickle.load(to_read)

In [5]:
hotels_df["Full review"].str.lower().str.contains("northern lights").value_counts()

False    110132
True       1136
Name: Full review, dtype: int64

In [6]:
hotels_df["Full review"].str.lower().str.contains("aurora").value_counts()

False    110122
True       1146
Name: Full review, dtype: int64

In [434]:
len(hotels_df["Property name"].unique())

1586

In [137]:
dropped_nas_df = test_df[test_df['Full review'].notna()]

/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [138]:
X = dropped_nas_df["Full review"]

/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [209]:
test_string = " ".join(X.values)

In [210]:
splits = test_string.split()

In [211]:
print(f'Number of strings: {len(splits)}')
print(f'Number of unique strings: {len(set(splits))}')

Number of strings: 6573298
Number of unique strings: 137350


In [212]:
freq_splits = FreqDist(splits)
print(f'10 most common strings: \n{freq_splits.most_common(10)}')

10 most common strings: 
[('the', 327077), ('and', 262089), ('a', 178218), ('to', 167244), ('was', 143116), ('in', 104112), ('The', 95227), ('of', 93531), ('is', 86696), ('I', 77086)]


In [213]:
short = set(s for s in splits if len(s) < 4)
short = [(s, freq_splits[s]) for s in short]
short.sort(key=lambda x:x[1], reverse=True)
short[:20]

[('the', 327077),
 ('and', 262089),
 ('a', 178218),
 ('to', 167244),
 ('was', 143116),
 ('in', 104112),
 ('The', 95227),
 ('of', 93531),
 ('is', 86696),
 ('I', 77086),
 ('for', 70917),
 ('we', 59447),
 ('We', 53331),
 ('at', 47891),
 ('our', 42454),
 ('on', 41281),
 ('had', 38852),
 ('but', 38157),
 ('it', 37969),
 ('are', 37896)]

In [214]:
long = set(s for s in splits if len(s) > 15)
long = [(s, freq_splits[s]) for s in long]
long.sort(key=lambda x:x[1], reverse=True)
long[:20]

[('recommendations.', 110),
 ('well-maintained.', 54),
 ('recommendations,', 37),
 ('air-conditioning', 23),
 ('professionalism.', 20),
 ('once-in-a-lifetime', 20),
 ('well-maintained,', 19),
 ('enthusiastically', 17),
 ('shampoo/conditioner', 12),
 ('miscommunication', 11),
 ('restaurant/lounge', 11),
 ('bathrooms/showers', 10),
 ('bed-and-breakfast', 10),
 ('air-conditioning,', 10),
 ('afternoon/evening', 10),
 ('reasonably-priced', 9),
 ('shops/restaurants', 8),
 ('air-conditioning.', 8),
 ('refrigerator/freezer,', 7),
 ('Wrangell-St.Elias', 7)]

In [16]:
def summarize(pattern, strings, freq):
    '''Summarize strings matching a pattern.
    '''
    # Find matches
    compiled_pattern = re.compile(pattern)
    matches = [s for s in strings if compiled_pattern.search(s)]
    
    # Print volume and proportion of matches
    print("{} strings which is {:.2%} of total".format(len(matches), len(matches) / len(strings)))
    
    # Create list of tuples containing matches and their frequency
    output = [(s, freq[s]) for s in set(matches)]
    output.sort(key=lambda x:x[1], reverse=True)
    
    return output


summarize(r"(/n)", splits, freq_splits)

NameError: name 'splits' is not defined

In [215]:
summarize(r"\d", splits, freq_splits)

59585 strings, that is 0.91% of total


[('2', 7259),
 ('3', 4597),
 ('5', 3006),
 ('4', 2788),
 ('10', 1848),
 ('1', 1676),
 ('6', 1291),
 ('15', 931),
 ('8', 904),
 ('7', 866),
 ('20', 845),
 ('30', 640),
 ('2nd', 626),
 ('24', 505),
 ('3rd', 481),
 ('12', 452),
 ('9', 436),
 ('11', 372),
 ('4th', 358),
 ('1st', 317),
 ('1/2', 295),
 ('1.', 252),
 ('2.', 250),
 ('45', 226),
 ('3.', 210),
 ('5th', 209),
 ('360', 207),
 ('14', 200),
 ('40', 197),
 ('100', 188),
 ('50', 182),
 ('25', 172),
 ('(2', 157),
 ('$10', 155),
 ('4.', 154),
 ('48', 146),
 ('$20', 143),
 ('13', 141),
 ('$200', 136),
 ('11th', 134),
 ('18', 133),
 ('2015.', 132),
 ('5.', 128),
 ('2013.', 128),
 ('24/7', 127),
 ('16', 123),
 ('$5', 122),
 ('2014.', 121),
 ('10-15', 121),
 ('200', 120),
 ('2012.', 119),
 ('#1', 112),
 ('90', 111),
 ('100%', 109),
 ('48.', 107),
 ('1)', 105),
 ('$100', 102),
 ('2)', 94),
 ('2014', 93),
 ('6.', 92),
 ('2013', 91),
 ('17', 91),
 ('8th', 90),
 ('80', 89),
 ('60', 89),
 ('6th', 88),
 ('$30', 87),
 ('2012', 86),
 ('2015', 84),


In [216]:
summarize(r"\w+-+\w+", splits, freq_splits)

19937 strings, that is 0.30% of total


, 239),
 ('Wi-Fi', 220),
 ('Check-in', 220),
 ('on-site', 184),
 ('in-room', 161),
 ('check-in.', 150),
 ('non-smoking', 125),
 ('check-out', 122),
 ('10-15', 121),
 ('check-in,', 115),
 ('well-appointed', 94),
 ('5-10', 81),
 ('5-star', 80),
 ('walk-in', 78),
 ('pick-up', 76),
 ('15-20', 75),
 ('wi-fi,', 67),
 ('made-to-order', 67),
 ('wake-up', 66),
 ('wi-fi.', 64),
 ('2-3', 63),
 ('pre-cruise', 63),
 ('well-stocked', 62),
 ('3-4', 62),
 ('Wi-fi', 60),
 ('mini-fridge', 58),
 ('Wrangell-St.', 56),
 ('well-maintained.', 54),
 ('black-out', 51),
 ('in-house', 50),
 ('top-notch', 48),
 ('well-maintained', 47),
 ('go-to', 47),
 ('pull-out', 46),
 ('Wi-Fi,', 45),
 ('well-equipped', 44),
 ('mid-town', 44),
 ('self-serve', 44),
 ('Wal-Mart', 44),
 ('full-size', 43),
 ('two-night', 43),
 ('well-appointed.', 43),
 ('off-season', 41),
 ('4-5', 41),
 ('mini-fridge,', 39),
 ('high-end', 39),
 ('1-2', 38),
 ('e-mail', 37),
 ('king-sized', 36),
 ('two-week', 36),
 ('24-hour', 36),
 ('snow-capped', 

In [217]:
def find_outlaw(word):
    """Find words that contain a same character 3+ times in a row.
    """
    is_outlaw = False
    for i, letter in enumerate(word):
        if i > 1:
            if word[i] == word[i-1] == word[i-2] and word[i].isalpha():
                is_outlaw = True
                break
    return is_outlaw
outlaws = [s for s in splits if find_outlaw(s)]
print("{} strings which is {:.2%} of total".format(len(outlaws), len(outlaws) / len(splits)))
outlaw_freq = [(s, freq_splits[s]) for s in set(outlaws)]
outlaw_freq.sort(key=lambda x:x[1], reverse=True)
outlaw_freq

558 strings which is 0.01% of total


[('AAA', 67),
 ('sooo', 50),
 ('soooo', 41),
 ('sooooo', 19),
 ('xxx', 9),
 ('soooooo', 8),
 ('SOOOO', 5),
 ('stafff', 5),
 ('LLL', 4),
 ('AAA.', 4),
 ('SOOO', 4),
 ('Sooo', 4),
 ('sooooooo', 4),
 ('looooong', 3),
 ('Sooooo', 3),
 ('Hmmm.', 3),
 ('Hmmmm.', 3),
 ('Hmmm', 3),
 ('waaaay', 3),
 ('alll', 3),
 ('toooo', 3),
 ('roooms', 3),
 ('Soooo', 3),
 ('fullly', 2),
 ('especiallly', 2),
 ('freee', 2),
 ('excelllent', 2),
 ('tooo', 2),
 ('coffeee', 2),
 ('sloooow.', 2),
 ('AAA,', 2),
 ('weee', 2),
 ('terrrible.', 2),
 ('SOOOOOO', 2),
 ('loooong', 2),
 ('juuuuust', 2),
 ('arrrived', 2),
 ('recommmend', 2),
 ('Princesss', 2),
 ('SSS', 2),
 ('Ahhh,', 1),
 ('MMMMM......', 1),
 ('AAAARRRRRRGGGGG...Embassy', 1),
 ('mmmmmm.', 1),
 ('Loooong', 1),
 ('Mmmmm!!!', 1),
 ("zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz's",
  1),
 ('REALLLY', 1),
 ('slooooow', 1),
 ('www.bringfido.com', 1),
 ('sooooosmall.', 1),
 ('(grrrrr).', 1),
 ('Hyattt', 1),
 ('recccomend', 1),
 ('hmmm...

In [53]:
tokeniser = RegexpTokenizer("[A-Za-z]+")
tokens = tokeniser.tokenize(test_string)
print(f'Number of tokens: {len(tokens)}')
print(f'Number of unique tokens: {len(set(tokens))}')

Number of tokens: 3042034
Number of unique tokens: 32323


In [218]:
stop_words = stopwords.words("english")
print(f'There are {len(stop_words)} stopwords.')
print(stop_words)

There are 179 stopwords.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own'

# Northern Lights only

## Creating nl_df

In [64]:
nl_df = dropped_nas_df[(dropped_nas_df["Full review"].str.lower().str.contains("northern lights")) | (dropped_nas_df["Full review"].str.lower().str.contains("aurora"))]

In [ ]:
nl_df = nl_df[nl_df["Lat, long"].notna()]

## EDA

In [65]:
nl_X = nl_df["Full review"]

In [66]:
nl_string = " ".join(nl_X.values)
nl_splits = nl_string.split()

In [24]:
print(f'Number of strings: {len(nl_splits)}')
print(f'Number of unique strings: {len(set(nl_splits))}')

Number of strings: 231013
Number of unique strings: 18603


In [26]:
freq_nl_splits = FreqDist(nl_splits)
print("100 most common strings:\n")
for i, term in enumerate(freq_nl_splits.most_common(100)):
    print(f'{i+1}. {term}')

100 most common strings:

1. ('the', 12551)
2. ('and', 8758)
3. ('to', 6466)
4. ('a', 5944)
5. ('was', 4157)
6. ('in', 3539)
7. ('of', 3481)
8. ('The', 2929)
9. ('is', 2927)
10. ('for', 2621)
11. ('we', 2430)
12. ('I', 2310)
13. ('We', 2093)
14. ('were', 2036)
15. ('with', 1792)
16. ('at', 1733)
17. ('you', 1572)
18. ('our', 1498)
19. ('on', 1397)
20. ('very', 1379)
21. ('are', 1346)
22. ('it', 1272)
23. ('that', 1208)
24. ('from', 1206)
25. ('had', 1182)
26. ('but', 1145)
27. ('room', 1065)
28. ('have', 1002)
29. ('this', 989)
30. ('see', 946)
31. ('so', 909)
32. ('as', 840)
33. ('us', 836)
34. ('great', 817)
35. ('not', 809)
36. ('stayed', 775)
37. ('Aurora', 768)
38. ('Northern', 760)
39. ('stay', 737)
40. ('place', 692)
41. ('up', 684)
42. ('there', 679)
43. ('hotel', 669)
44. ('an', 658)
45. ('they', 642)
46. ('be', 633)
47. ('lights', 632)
48. ('out', 627)
49. ('all', 604)
50. ('night', 595)
51. ('It', 580)
52. ('if', 564)
53. ('my', 563)
54. ('northern', 547)
55. ('one', 528)
56

In [27]:
summarize(r"(/n)", nl_splits, freq_nl_splits)

14 strings which is 0.01% of total


[('day/night', 3),
 ("aurora's/northern", 1),
 ('$120/night', 1),
 ('wake-up/notification', 1),
 ('$50/night', 1),
 ('$85/night,', 1),
 ('town//airports/northern', 1),
 ('$100/nt,', 1),
 ('$249.00/night', 1),
 ('$64/night', 1),
 ('$525/night', 1),
 ('day/night,', 1)]

In [28]:
summarize(r"\d", nl_splits, freq_nl_splits)

2263 strings which is 0.98% of total


[('2', 259),
 ('3', 187),
 ('4', 152),
 ('5', 119),
 ('10', 54),
 ('20', 52),
 ('1', 47),
 ('6', 45),
 ('15', 41),
 ('30', 40),
 ('7', 31),
 ('2nd', 27),
 ('3rd', 26),
 ('24', 19),
 ('8', 17),
 ('9', 16),
 ('45', 14),
 ('40', 12),
 ('25', 11),
 ('11', 11),
 ('1st', 11),
 ('2015.', 11),
 ('1/2', 9),
 ('-20', 9),
 ('2am', 9),
 ('2012.', 8),
 ('2014.', 8),
 ('2012', 8),
 ('35', 8),
 ('100', 7),
 ('14', 7),
 ('2014', 7),
 ('10pm', 7),
 ('$5', 7),
 ('2:00', 6),
 ('4th', 6),
 ('16', 6),
 ('2013.', 6),
 ('60', 6),
 ('$100', 6),
 ('12', 6),
 ('2015', 6),
 ('2:30', 6),
 ('4am', 5),
 ('2016.', 5),
 ('18', 5),
 ('24/7.', 5),
 ('48.', 5),
 ('11pm', 5),
 ('(5', 5),
 ('$25', 5),
 ('$30', 5),
 ('13', 5),
 ('10:30', 4),
 ('48', 4),
 ('8th', 4),
 ('50', 4),
 ('7th', 4),
 ('1am', 4),
 ('300', 4),
 ('360', 4),
 ('2013', 4),
 ('$20', 4),
 ('2008.', 4),
 ('1-2', 4),
 ('2017.', 4),
 ('11:30', 4),
 ('2013,', 4),
 ('10-15', 4),
 ('#1', 4),
 ('2020', 4),
 ('2019', 4),
 ('9:30', 4),
 ('2.', 4),
 ('0', 3),
 ('1.

In [29]:
summarize(r"\w+-+\w+", nl_splits, freq_nl_splits)

760 strings which is 0.33% of total


[('wake-up', 38),
 ('check-in', 18),
 ('walkie-talkie', 13),
 ('wi-fi', 7),
 ('well-equipped', 7),
 ('check-in,', 6),
 ('north-facing', 5),
 ('flight-seeing', 5),
 ('Check-in', 4),
 ('pick-up', 4),
 ('1-2', 4),
 ('10-15', 4),
 ('cross-country', 4),
 ('in-room', 4),
 ('walkie-talkies', 4),
 ('Wi-Fi', 3),
 ('5-star', 3),
 ('full-sized', 3),
 ('in-house', 3),
 ('last-minute', 3),
 ('off-season', 3),
 ('Covid-19', 3),
 ('check-in.', 3),
 ('3-night', 3),
 ('on-site', 3),
 ('2-3', 3),
 ('dog-sledding', 3),
 ('self-serve', 3),
 ('flat-screen', 3),
 ('round-trip', 3),
 ('pre-cruise', 2),
 ('full-size', 2),
 ('non-stop', 2),
 ('hands-down', 2),
 ('make-your-own', 2),
 ('once-in-a-lifetime', 2),
 ('off-season.', 2),
 ('15-20', 2),
 ('king-size', 2),
 ('e-mail,', 2),
 ('mini-fridge', 2),
 ('dog-sled', 2),
 ('so-so', 2),
 ('high-end', 2),
 ('mid-century', 2),
 ('warm-up', 2),
 ('set-up', 2),
 ('to-go', 2),
 ('high-tech', 2),
 ('Wi-fi', 2),
 ('wi-fi.', 2),
 ('non-smoking', 2),
 ('sky-cams,', 2),
 (

In [30]:
summarize(r"\w+--+\w+", nl_splits, freq_nl_splits)

59 strings which is 0.03% of total


[('down--so', 1),
 ('wi-fi---we', 1),
 ('host--incredibly', 1),
 ('city--close', 1),
 ('sorry--we', 1),
 ('lights--which', 1),
 ('out--it', 1),
 ('food--and', 1),
 ('background--a', 1),
 ('late--however,', 1),
 ('distance--they', 1),
 ('off--a', 1),
 ('visit--a', 1),
 ('fresh--bed', 1),
 ('airport--I', 1),
 ('location--strolled', 1),
 ('borrow--coats,', 1),
 ('book--buy', 1),
 ('TelAlaska--very', 1),
 ('out--best', 1),
 ('wi-fi--too', 1),
 ('internet--but', 1),
 ('maker--all', 1),
 ('Note--below:', 1),
 ('it--once', 1),
 ('night--I', 1),
 ('way--clean,', 1),
 ('disappointed--seeing', 1),
 ('ask--we', 1),
 ('lights--nothing', 1),
 ('matter---they', 1),
 ('ride--the', 1),
 ("Fairbanks--they'll", 1),
 ('places--it', 1),
 ('bucket--shooting', 1),
 ('cozy--very', 1),
 ('again--perhaps', 1),
 ('ride--what', 1),
 ('street--be', 1),
 ('good----but', 1),
 ('draw--on', 1),
 ('river--a', 1),
 ("AMAZING--I'm", 1),
 ('scrumptious--took', 1),
 ('clean--what', 1),
 ('trip--The', 1),
 ('memorable--fir

In [417]:
long = set(s for s in splits if len(s) > 15)
long = [(s, freq_nl_splits[s]) for s in long]
long.sort(key=lambda x:x[1], reverse=True)
long[:20]

[('recommendations.', 5),
 ('recommendations,', 2),
 ('once-in-a-lifetime', 2),
 ('goalaskatours.com', 2),
 ('impression/expectations', 1),
 ('warm,beautifully', 1),
 ('lights....though', 1),
 ('weather-pending,', 1),
 ('windows....perfect', 1),
 ('accordingly.....', 1),
 ('host--incredibly', 1),
 ('attractions/things', 1),
 ('http://www.auroracabin.com/OvernightEnglishOne.html', 1),
 ('downtown..thanks', 1),
 ('unproffessional.', 1),
 ('dissatisfaction,', 1),
 ('expedition-style', 1),
 ('settings/exposures.', 1),
 ('fruit/cereal/danishes,', 1),
 ('friendly/hospitable', 1)]

In [227]:
def find_misspelled(word, threshold=85):

    chars = string.ascii_letters
    for item in itertools.product(chars, repeat=len(word)):
        fuzz.ratio(word, itm)
fuzz.ratio("northern lights", "nartheen lights")

## pyLDAvis analysis

In [391]:
nl_df = dropped_nas_df[(dropped_nas_df["Full review"].str.lower().str.contains("northern lights")) | (dropped_nas_df["Full review"].str.lower().str.contains("aurora"))]

In [392]:
lats = []
longs = []
for i in range(nl_df.shape[0]):
    lats.append(float(nl_df.iloc[i]["Lat, long"].split(",")[0]))
    longs.append(float(nl_df.iloc[i]["Lat, long"].split(",")[1]))
nl_df["Latitude"] = lats
nl_df["Longitude"] = longs
nl_df = nl_df[nl_df["Lat, long"].notna()]

In [393]:
lats = []
longs = []
for i in range(nl_df.shape[0]):
    lats.append(float(nl_df.iloc[i]["Lat, long"].split(",")[0]))
    longs.append(float(nl_df.iloc[i]["Lat, long"].split(",")[1]))
nl_df["Latitude"] = lats
nl_df["Longitude"] = longs

In [399]:
len(test_df["Property name"].unique())

687

In [444]:
mids = set(s for s in splits if len(s) > 5)
mids = [(s, freq_splits[s]) for s in mids]
mids.sort(key=lambda x:x[1], reverse=True)
mids[:100]

[('stayed', 735),
 ('Northern', 724),
 ('Aurora', 723),
 ('lights', 608),
 ('northern', 529),
 ('aurora', 467),
 ('breakfast', 442),
 ('nights', 422),
 ('Lights', 387),
 ('Fairbanks', 364),
 ('comfortable', 333),
 ('Alaska', 325),
 ('really', 324),
 ('friendly', 271),
 ('wonderful', 264),
 ('during', 253),
 ('location', 239),
 ('viewing', 223),
 ('beautiful', 215),
 ('Denali', 212),
 ('lights.', 199),
 ('around', 198),
 ('recommend', 198),
 ("didn't", 189),
 ('helpful', 188),
 ('enough', 183),
 ('outside', 182),
 ('enjoyed', 181),
 ('service', 177),
 ('Bettles', 174),
 ('experience', 173),
 ('amazing', 173),
 ('little', 168),
 ('Borealis', 166),
 ('Lights.', 164),
 ('because', 164),
 ('perfect', 163),
 ('staying', 159),
 ('everything', 158),
 ('restaurant', 150),
 ('clean,', 148),
 ('family', 147),
 ('definitely', 143),
 ('guests', 140),
 ('people', 139),
 ('before', 138),
 ('coffee', 137),
 ('comfortable.', 136),
 ('night.', 131),
 ('Fairbanks.', 130),
 ('better', 130),
 ('kitchen', 1

In [446]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle

class aurora_nlp_pipe:
   
    def __init__(self, vectorizer=CountVectorizer(stop_words="english", min_df=5), tokenizer=None, cleaning_function=None, 
                 stemmer=None, model=None):
        """
        A class for pipelining our data in NLP problems. The user provides a series of 
        tools, and this class manages all of the training, transforming, and modification
        of the text data.
        ---
        Inputs:
        vectorizer: the model to use for vectorization of text data
        tokenizer: The tokenizer to use, if none defaults to split on spaces
        cleaning_function: how to clean the data, if None, defaults to the in built class
        """
        if not tokenizer:
            tokenizer = self.splitter
        if not cleaning_function:
            cleaning_function = self.clean_text
        self.stemmer = stemmer
        self.tokenizer = tokenizer
        self.model = model
        self.cleaning_function = cleaning_function
        self.vectorizer = vectorizer
        self._is_fit = False
        
    def splitter(self, text):
        """
        Default tokenizer that splits on spaces naively
        """
        return text.split(' ')
        
    def clean_text(self, text, tokenizer, stemmer):
        """
        A naive function to lowercase all works can clean them quickly.
        This is the default behavior if no other cleaning function is specified
        """
        cleaned_text = []
        for review in text:
            review = re.sub('(\W?)(\d+)(\S?)((\d+)?)(((a|p)[m])|((\s)((a|p)[m]))?)', ' ', review)
            review = re.sub('(?!\w+|\s)--+(?=\s|\w+)', ' ', review)
            review = re.sub('(?!\s)((\S*)((.com)|(.com\/))(\S*))', ' ', review)
            review = re.sub('(?!\w+)([,]+|[.][.]+|\/+)(?=\w+)', ' ', review)
            review = re.sub('([A-Z]([a-z]+))((\s[A-Z]([a-z]+))+)', ' ', review)
            
            for word in review.split(' '):
                if word 
            

            cleaned_words = []
            for word in tokenizer(post):
                low_word = word.lower()
                if stemmer:
                    low_word = stemmer.stem(low_word)
                cleaned_words.append(low_word)
            cleaned_text.append(' '.join(cleaned_words))
        return cleaned_text
    
    def fit(self, text):
        """
        Cleans the data and then fits the vectorizer with
        the user provided text
        """
        clean_text = self.cleaning_function(text, self.tokenizer, self.stemmer)
        self.vectorizer.fit(clean_text)
        self._is_fit = True
        
    def transform(self, text):
        """
        Cleans any provided data and then transforms the data into
        a vectorized format based on the fit function. Returns the
        vectorized form of the data.
        """
        if not self._is_fit:
            raise ValueError("Must fit the models before transforming!")
        clean_text = self.cleaning_function(text, self.tokenizer, self.stemmer)
        return self.vectorizer.transform(clean_text)
    
    def save_pipe(self, filename):
        """
        Writes the attributes of the pipeline to a file
        allowing a pipeline to be loaded later with the
        pre-trained pieces in place.
        """
        if type(filename) != str:
            raise TypeError("filename must be a string")
        pickle.dump(self.__dict__, open(filename+".mdl", 'wb'))
        
    def load_pipe(self, filename):
        """
        Writes the attributes of the pipeline to a file
        allowing a pipeline to be loaded later with the
        pre-trained pieces in place.
        """
        if type(filename) != str:
            raise TypeError("filename must be a string")
        if filename[-4:] != '.mdl':
            filename += '.mdl'
        self.__dict__ = pickle.load(open(filename, 'rb'))

In [42]:
test_text = "I went to Alaska for 7 days to have a great time ang uess what I did I woke every day at 6:30am to see the wildlife (cariboo) eating outside my window."
test_text = re.sub('(\W?)(\d+)(\S?)((\d+)?)(((a|p)[m])|((\s)((a|p)[m]))?)', ' ', test_text)
test_text

'I went to Alaska for  days to have a great time ang uess what I did I woke every day at  to see the wildlife (cariboo) eating outside my window.'

In [52]:
test_text = "I went to Alaska for 7 days to have a great time ang uess what I did---- I woke every day at 6:30am to see the wildlife (cariboo) eating outside my window. The last/only time I didn't like the trip was when I,had to visit the website alaska.com to find a place to eat.I recommmend Lakeshore Inn & Suites!"
test_text = re.sub('(\W?)(\d+)(\S?)((\d+)?)(((a|p)[m])|((\s)((a|p)[m]))?)', ' ', test_text)
test_text = re.sub('(?!\w+|\s)--+(?=\s|\w+)', ' ', test_text)
test_text = re.sub('(?!\s)((\S*)(([.]com)|([.]com\/))(\S*))', ' ', test_text)
test_text = re.sub('(?!\w+)([,]+|[.][.]+|\/+)(?=\w+)', ' ', test_text)
test_text = re.sub('([A-Z]([a-z]+))((\s[A-Z]([a-z]+))+)', ' ', test_text)
test_text

"I went to Alaska for  days to have a great time ang uess what I did  I woke every day at  to see the wildlife (cariboo) eating outside my window. The last only time I didn't like the trip was when I had to visit the website   to find a place to eat.I recommmend   & Suites!"

In [254]:
vectorizer = CountVectorizer(stop_words="english", lowercase=True)

In [255]:
nl_vectorized = vectorizer.fit_transform(nl_X.values)

In [256]:
pd.DataFrame(nl_vectorized.A, columns=vectorizer.get_feature_names())

,00,000,00a,00am,00pm,01,0100,03,06,0800,...,往阿拉斯加大學,或上快速道路,房間寬敞乾淨,旁邊有兩家大超市,是非常好的選擇,水周りも清潔,私にとってここはその中でもトップクラスの宿です,若喜歡有獨立空間的旅客,郊外に行かずにオーロラ鑑賞をするなら絶好のロケーションです,都非常快速便捷
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1589,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1590,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1591,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [394]:
nl_df.to_csv("Northern lights reviews with lat and long.csv")

# First pyLDAvis

In [91]:
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn

In [ ]:
vectorizer = CountVectorizer(stop_words="english", lowercase=True)

In [376]:
vectorizer = CountVectorizer(stop_words = 'english', lowercase = True, max_df = 0.5)
nl_df_vectorized = vectorizer.fit_transform(nl_X.values)
nl_lda = LatentDirichletAllocation(n_components=20, random_state=0)
nl_lda.fit(nl_df_vectorized)
pyLDAvis.sklearn.prepare(nl_lda, nl_df_vectorized, vectorizer)

LatentDirichletAllocation(n_components=20, random_state=0)

In [377]:
pyLDAvis.sklearn.prepare(nl_lda, nl_df_vectorized, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.119394  0.041210       1        1  15.148298
1      0.025372 -0.083767       2        1  12.767359
6      0.141635  0.089517       3        1  10.965607
17     0.088042  0.057500       4        1   6.301313
4      0.012189 -0.130371       5        1   6.201910
9      0.046188 -0.019074       6        1   6.048457
16     0.037333 -0.026707       7        1   5.816735
10     0.029684 -0.043972       8        1   5.531884
2     -0.011361 -0.084783       9        1   4.323375
3     -0.012058 -0.077108      10        1   4.313265
14     0.004183 -0.045556      11        1   3.871037
18    -0.002443 -0.036331      12        1   3.303268
7      0.033026  0.057342      13        1   3.193635
19    -0.023281  0.045629      14        1   2.311419
13    -0.010576  0.063566      15        1   2.113196
8     -0.056040  0.020900      16        1   1.933397
0     -0.048684  0.099698      17        1   1.883656
15    -0.089240  0.021185      18        1   1.457306
12    -0.163847  0.083208      19        1   1.419623
11    -0.119515 -0.032087      20        1   1.095261, topic_info=             Term         Freq        Total Category  logprob  loglift
4310        lodge   952.000000   952.000000  Default  30.0000  30.0000
3601        hotel   832.000000   832.000000  Default  29.0000  29.0000
904       bettles   189.000000   189.000000  Default  28.0000  28.0000
6124         room  1240.000000  1240.000000  Default  27.0000  27.0000
4676          mok   230.000000   230.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
3599          hot     3.673901   233.403094  Topic20  -5.6179   0.3627
3261          got     3.404478   217.819353  Topic20  -5.6940   0.3556
4344          lot     3.170215   134.815121  Topic20  -5.7653   0.7641
1629  comfortable     3.270560   525.946319  Topic20  -5.7342  -0.5661
2177          did     3.220806   321.046260  Topic20  -5.7495  -0.0878

[1761 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.101000        00
0         2  0.101000        00
0         3  0.033667        00
0         4  0.033667        00
0         6  0.134667        00
...     ...       ...       ...
8088     18  0.606259    yearly
8101      6  0.739810   younger
8106     10  0.563729     yucks
8109     12  0.763361  yummiest
8112     10  0.563729       zac

[5756 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 2, 7, 18, 5, 10, 17, 11, 3, 4, 15, 19, 8, 20, 14, 9, 1, 16, 13, 12])

# Second pyLSAvis

In [378]:
vectorizer = CountVectorizer(stop_words = 'english', lowercase = True, max_df = 0.5)
nl_df_vectorized = vectorizer.fit_transform(nl_X.values)
nl_lda = LatentDirichletAllocation(n_components=10, random_state=0)
nl_lda.fit(nl_df_vectorized)

LatentDirichletAllocation(random_state=0)

In [379]:
pyLDAvis.sklearn.prepare(nl_lda, nl_df_vectorized, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.007368 -0.066316       1        1  19.822348
5     -0.098135 -0.028348       2        1  18.612082
6     -0.162341  0.011947       3        1  17.786067
9     -0.019823 -0.032465       4        1  10.882873
3      0.030233 -0.058469       5        1   8.247684
2      0.096426 -0.065370       6        1   7.028834
4      0.080896 -0.039063       7        1   6.677365
7     -0.017854  0.054442       8        1   4.294962
0      0.062103  0.158164       9        1   3.327060
8      0.021127  0.065478      10        1   3.320726, topic_info=             Term         Freq        Total Category  logprob  loglift
4310        lodge   943.000000   943.000000  Default  30.0000  30.0000
3601        hotel   831.000000   831.000000  Default  29.0000  29.0000
904       bettles   180.000000   180.000000  Default  28.0000  28.0000
1278          car   144.000000   144.000000  Default  27.0000  27.0000
3300        great  1005.000000  1005.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
2546      enjoyed     8.716260   177.538635  Topic10  -5.8631   0.3910
3546         home     8.800743   197.257670  Topic10  -5.8535   0.2953
1629  comfortable    10.207570   526.702849  Topic10  -5.7052  -0.5385
3601        hotel    10.524119   831.670949  Topic10  -5.6746  -0.9648
7321         time     8.885435   418.669418  Topic10  -5.8439  -0.4477

[849 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
49        1  0.177580        16
49        4  0.088790        16
49        7  0.355160        16
49        9  0.355160        16
60        3  0.829965        19
...     ...       ...       ...
8087      8  0.013243      year
8087      9  0.039728      year
8087     10  0.039728      year
8097      4  0.824221   yoghurt
8109      5  0.745193  yummiest

[2319 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 7, 10, 4, 3, 5, 8, 1, 9])

## Pickling back

In [443]:
with open("alaska_hotels_first_attempt.pickle", "wb") as to_write:
    pickle.dump(test_df, to_write)

In [ ]:
import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# lookup suggestions for single-word input strings
input_term = "memebers"  # misspelling of "members"
# max edit distance per lookup
# (max_edit_distance_lookup <= max_dictionary_edit_distance)
suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST,
                               max_edit_distance=2)
# display suggestion term, term frequency, and edit distance
for suggestion in suggestions:
    print(suggestion)

## Testing cleaning function (while still building it)

In [85]:
def clean_text(text):

    cleaned_text = []
    for review in text:
        review = review.lower()
        review = re.sub('(?!\s)((\S*)((.com)|(.com\/))(\S*))', ' ', review)
        review = re.sub('[^A-Za-z\s-]', '', review)
        review = re.sub('(\W?)(\d+)(\S?)((\d+)?)(((a|p)[m])|((\s)((a|p)[m]))?)', ' ', review)
        review = re.sub('(?!\w+|\s)--+(?=\s|\w+)', ' ', review)
        review = re.sub('(?!\w+)([,]+|[.][.]+|\/+)(?=\w+)', ' ', review)
        review = re.sub('([A-Z]([a-z]+))((\s[A-Z]([a-z]+))+)', ' ', review)
        review = " ".join([word for word in review.split() if word not in stopwords.words('english')])
        cleaned_text.append(review)

    return cleaned_text

In [84]:
clean_text(list(hotels_df["Full review"]))

['stay lots hotels including covid harbor right great hotel fair price excellent location right marina friendly staff clean rooms reasonable breakfast get details right ice buckets bags ice machine dont go room get ice pail return ice machine plenty used books night light built hair dryer holder bathroom beds get place stay',
 'girlfriend wanted get away town enjoyed great evening even covid restrictions enjoyed hotel also everything could serve great breakfast order pretty much anything want would make included would highly',
 'stayed twice last two trips seward views marina fantastic rooms spotless bathrooms large hotel good value rate adapted breakfast covid friendly mostly service great people friendly seem genuinely happy work hotel proud highly familes well first stay kids blast pool',
 'everything really nice almost great hadnt bed beds horrible take mattress one put top helped much back next year hopefully get better bed',
 'terrific place clean friendly staff excellent breakfa

In [88]:
nl_X = nl_df["Full review"]
nl_X = clean_text(nl_X)
nl_test_string = " ".join(nl_X)

nl_splits = nl_test_string.split()
print(f'Number of strings: {len(nl_splits)}')
print(f'Number of unique strings: {len(set(nl_splits))}')
nl_freq_splits = FreqDist(nl_splits)
print(f'10 most common strings: \n{nl_freq_splits.most_common(10)}')

Number of strings: 115559
Number of unique strings: 9230
10 most common strings: 
[('lights', 1630), ('aurora', 1513), ('room', 1419), ('northern', 1326), ('great', 1168), ('lodge', 1022), ('see', 969), ('stay', 966), ('us', 948), ('hotel', 922)]


In [89]:
for i, term in enumerate(nl_freq_splits.most_common(100)):
    print(f'{i+1}. {term}')

1. ('lights', 1630)
2. ('aurora', 1513)
3. ('room', 1419)
4. ('northern', 1326)
5. ('great', 1168)
6. ('lodge', 1022)
7. ('see', 969)
8. ('stay', 966)
9. ('us', 948)
10. ('hotel', 922)
11. ('stayed', 865)
12. ('night', 862)
13. ('place', 854)
14. ('breakfast', 699)
15. ('staff', 673)
16. ('fairbanks', 657)
17. ('good', 650)
18. ('one', 626)
19. ('nice', 625)
20. ('clean', 595)
21. ('nights', 593)
22. ('rooms', 572)
23. ('would', 561)
24. ('view', 540)
25. ('alaska', 522)
26. ('-', 521)
27. ('also', 491)
28. ('time', 473)
29. ('well', 445)
30. ('get', 433)
31. ('area', 431)
32. ('friendly', 428)
33. ('trip', 413)
34. ('even', 409)
35. ('back', 398)
36. ('food', 378)
37. ('could', 377)
38. ('go', 372)
39. ('two', 367)
40. ('helpful', 355)
41. ('best', 351)
42. ('really', 351)
43. ('location', 350)
44. ('day', 348)
45. ('wonderful', 344)
46. ('like', 338)
47. ('experience', 332)
48. ('viewing', 328)
49. ('first', 326)
50. ('beautiful', 319)
51. ('amazing', 293)
52. ('made', 287)
53. ('bor

In [92]:
vectorizer = CountVectorizer(stop_words = 'english', lowercase = True, max_df = 0.5)
nl_df_vectorized = vectorizer.fit_transform(nl_X)
nl_lda = LatentDirichletAllocation(n_components=10, random_state=0)
nl_lda.fit(nl_df_vectorized)

/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.021327  0.009350       1        1  26.238659
8     -0.032899  0.020087       2        1  14.556856
3     -0.057786 -0.013445       3        1  10.995366
7     -0.096364  0.040799       4        1  10.028762
6     -0.026976 -0.015616       5        1   9.747877
1     -0.036410 -0.019671       6        1   8.526320
2     -0.030325  0.003130       7        1   7.763605
9     -0.001727 -0.022776       8        1   7.322407
0      0.165263  0.119439       9        1   2.672951
5      0.138551 -0.121298      10        1   2.147198, topic_info=           Term         Freq        Total Category  logprob  loglift
3659      hotel   864.000000   864.000000  Default  30.0000  30.0000
4440      lodge   950.000000   950.000000  Default  29.0000  29.0000
6431       room  1309.000000  1309.000000  Default  28.0000  28.0000
955   breakfast   652.000000   65

In [93]:
pyLDAvis.sklearn.prepare(nl_lda, nl_df_vectorized, vectorizer)

/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.021327  0.009350       1        1  26.238659
8     -0.032899  0.020087       2        1  14.556856
3     -0.057786 -0.013445       3        1  10.995366
7     -0.096364  0.040799       4        1  10.028762
6     -0.026976 -0.015616       5        1   9.747877
1     -0.036410 -0.019671       6        1   8.526320
2     -0.030325  0.003130       7        1   7.763605
9     -0.001727 -0.022776       8        1   7.322407
0      0.165263  0.119439       9        1   2.672951
5      0.138551 -0.121298      10        1   2.147198, topic_info=           Term         Freq        Total Category  logprob  loglift
3659      hotel   864.000000   864.000000  Default  30.0000  30.0000
4440      lodge   950.000000   950.000000  Default  29.0000  29.0000
6431       room  1309.000000  1309.000000  Default  28.0000  28.0000
955   breakfast   652.000000   65

## Generator

In [98]:
def aurora_review_printer():
    aurora_reviews = list(nl_df["Full review"])
    length = len(aurora_reviews)
    i = 0
    while i < length:
        yield aurora_reviews[i]
        i += 1

review_gen = aurora_review_printer()

/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [131]:
next(review_gen)

/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"Always a great pleasure to stay at Pikes.  We're looking forward to the installation of the new Aurora Conservatory - we'll be back this winter to check it out!  Our family had a delightful repast with your Smore's kit around the outdoor fireplace.  "

In [165]:
hotels_df

/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Property name,Property address,"Lat, long",Reviewer,Review date,Date of stay,Review title,Full review,Review link
0,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",John M,October 2020,October 2020,Awesome hotel doing all the right things,I stay at lots of hotels (including during Cov...,https://www.tripadvisor.com/ShowUserReviews-g6...
1,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",Kenai Drift Anglers,October 2020,October 2020,Great Local Get Away,My girlfriend just wanted to get away from tow...,https://www.tripadvisor.com/ShowUserReviews-g6...
2,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",TravelGirl2006,October 2020,October 2020,We love this hotel,Have stayed here twice on our last two trips t...,https://www.tripadvisor.com/ShowUserReviews-g6...
3,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",maduro44,October 2020,September 2020,Bed,Everything was really nice almost great if it ...,https://www.tripadvisor.com/ShowUserReviews-g6...
4,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",rtfields2020,October 2020,September 2020,Great place,"Terrific place to stay. Comfortable and clean,...",https://www.tripadvisor.com/ShowUserReviews-g6...
...,...,...,...,...,...,...,...,...,...
111525,Bootlegger Cove House,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111526,Westwood House,"4061 Westwood Dr, Anchorage, AK 99517-1055",NaN,NaN,NaN,NaN,NaN,NaN,NaN
111527,Talkeetna Fireweed Cabins,"25683 Talkeetna Spur Road, Talkeetna, AK 99676",NaN,NaN,NaN,NaN,NaN,NaN,NaN
111528,Athenian Village,"3725 E 43rd Pl, Anchorage, AK 99508-5529",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
updated_early_df

/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Property name,Property address,"Lat, long",Reviewer,Review date,Date of stay,Review title,Full review,Review link
0,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",John M,October 2020,October 2020,Awesome hotel doing all the right things,I stay at lots of hotels (including during Cov...,https://www.tripadvisor.com/ShowUserReviews-g6...
1,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",Kenai Drift Anglers,October 2020,October 2020,Great Local Get Away,My girlfriend just wanted to get away from tow...,https://www.tripadvisor.com/ShowUserReviews-g6...
2,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",TravelGirl2006,October 2020,October 2020,We love this hotel,Have stayed here twice on our last two trips t...,https://www.tripadvisor.com/ShowUserReviews-g6...
3,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",maduro44,October 2020,September 2020,Bed,Everything was really nice almost great if it ...,https://www.tripadvisor.com/ShowUserReviews-g6...
4,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",rtfields2020,October 2020,September 2020,Great place,"Terrific place to stay. Comfortable and clean,...",https://www.tripadvisor.com/ShowUserReviews-g6...
...,...,...,...,...,...,...,...,...,...
50455,Hatcher Pass Bed & Breakfast,"9000 N Palmer Fishhook Rd, Palmer, AK 99645-8043","61.686283,-149.241776",Lucyinsky,September 2007,September 2007,I Loved it Here!!,"My husband, parents, and I stayed here for two...",https://www.tripadvisor.com/ShowUserReviews-g3...
50456,Hatcher Pass Bed & Breakfast,"9000 N Palmer Fishhook Rd, Palmer, AK 99645-8043","61.686283,-149.241776",fsctb5,March 2007,March 2007,"Great Location, Very Cozy.","March, 2007 When we first pulled up to Hatcher...",https://www.tripadvisor.com/ShowUserReviews-g3...
50457,Hatcher Pass Bed & Breakfast,"9000 N Palmer Fishhook Rd, Palmer, AK 99645-8043","61.686283,-149.241776",LaughALot99,August 2006,August 2006,"Cozy and warming feeling, very clean",This was our second B&B. The first one in Sew...,https://www.tripadvisor.com/ShowUserReviews-g3...
50458,Hatcher Pass Bed & Breakfast,"9000 N Palmer Fishhook Rd, Palmer, AK 99645-8043","61.686283,-149.241776",meginoregon,May 2006,NaN,Super Nice,"We stayed at Hatcher Pass B&B in July 2005, an...",https://www.tripadvisor.com/ShowUserReviews-g3...


In [7]:
all_hotels_df = pd.concat([updated_early_df, hotels_df.iloc[67214:]])

In [8]:
all_hotels_df

,Property name,Property address,"Lat, long",Reviewer,Review date,Date of stay,Review title,Full review,Review link
0,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",John M,October 2020,October 2020,Awesome hotel doing all the right things,I stay at lots of hotels (including during Cov...,https://www.tripadvisor.com/ShowUserReviews-g6...
1,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",Kenai Drift Anglers,October 2020,October 2020,Great Local Get Away,My girlfriend just wanted to get away from tow...,https://www.tripadvisor.com/ShowUserReviews-g6...
2,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",TravelGirl2006,October 2020,October 2020,We love this hotel,Have stayed here twice on our last two trips t...,https://www.tripadvisor.com/ShowUserReviews-g6...
3,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",maduro44,October 2020,September 2020,Bed,Everything was really nice almost great if it ...,https://www.tripadvisor.com/ShowUserReviews-g6...
4,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",rtfields2020,October 2020,September 2020,Great place,"Terrific place to stay. Comfortable and clean,...",https://www.tripadvisor.com/ShowUserReviews-g6...
...,...,...,...,...,...,...,...,...,...
111525,Bootlegger Cove House,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111526,Westwood House,"4061 Westwood Dr, Anchorage, AK 99517-1055",NaN,NaN,NaN,NaN,NaN,NaN,NaN
111527,Talkeetna Fireweed Cabins,"25683 Talkeetna Spur Road, Talkeetna, AK 99676",NaN,NaN,NaN,NaN,NaN,NaN,NaN
111528,Athenian Village,"3725 E 43rd Pl, Anchorage, AK 99508-5529",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
with open("all_alaska_hotels_full.pickle", "wb") as to_write:
    pickle.dump(all_hotels_df, to_write)

In [11]:
dropped_nas_df = all_hotels_df[all_hotels_df['Full review'].notna()]
nl_df = dropped_nas_df[(dropped_nas_df["Full review"].str.lower().str.contains("northern lights")) | (dropped_nas_df["Full review"].str.lower().str.contains("aurora"))]
nl_X = list(nl_df["Full review"])

In [10]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
for i in topic.argsort()[:-no_top_words - 1:-1]]))


INSTALLED VERSIONS
------------------
commit           : 67a3d4241ab84419856b84fc3ebc9abcbe66c6b3
python           : 3.8.5.final.0
python-bits      : 64
OS               : Darwin
OS-release       : 19.6.0
Version          : Darwin Kernel Version 19.6.0: Thu Jun 18 20:49:00 PDT 2020; root:xnu-6153.141.1~1/RELEASE_X86_64
machine          : x86_64
processor        : i386
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : None.UTF-8

pandas           : 1.1.4
numpy            : 1.19.1
pytz             : 2020.1
dateutil         : 2.8.1
pip              : 20.2.3
setuptools       : 49.6.0.post20200917
Cython           : None
pytest           : 6.1.2
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : 1.3.6
lxml.etree       : 4.5.2
html5lib         : 1.1
pymysql          : None
psycopg2         : 2.8.6 (dt dec pq3 ext lo64)
jinja2           : 2.11.2
IPython          : 7.18.1
pandas_dataread

## Whole corpus

In [140]:
X = dropped_nas_df["Full review"]
X = clean_text(X.values)
test_string = " ".join(X)
splits = test_string.split()

print(f'Number of strings: {len(splits)}')
print(f'Number of unique strings: {len(set(splits))}')
freq_splits = FreqDist(splits)
for i, term in enumerate(freq_splits.most_common(100)):
    print(f'{i+1}. {term}')

/opt/anaconda3/envs/metis/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Number of strings: 5113684
Number of unique strings: 74524
1. ('room', 81140)
2. ('hotel', 59980)
3. ('stay', 49448)
4. ('great', 48675)
5. ('clean', 39875)
6. ('staff', 39027)
7. ('breakfast', 38346)
8. ('stayed', 38065)
9. ('us', 35365)
10. ('nice', 35277)
11. ('rooms', 35051)
12. ('good', 34378)
13. ('place', 34004)
14. ('one', 31849)
15. ('would', 30153)
16. ('night', 24517)
17. ('friendly', 24134)
18. ('alaska', 21687)
19. ('lodge', 21427)
20. ('well', 20936)
21. ('time', 20800)
22. ('-', 20236)
23. ('location', 19805)
24. ('area', 19328)
25. ('also', 18591)
26. ('get', 18334)
27. ('day', 18265)
28. ('view', 17996)
29.